In [ ]:
"""
install：
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh
bash Miniforge3-MacOSX-arm64.sh

conda create -n cnn-mac python=3.12
conda activate cnn-mac
pip install tensorflow-macos tensorflow-metal numpy pandas opencv-python scikit-image scikit-learn
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
"""

In [1]:
# Import required libraries
import os
from skimage.io import imread, imsave
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

print("Load environment successfully")

# Enable GPU memory growth for TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Create a new folder to store the flipped images
os.makedirs('train_ims_rever', exist_ok=True)

# Get all image names
image_names = os.listdir('train_ims')

# Flip images horizontally, add prefix and save them to the new folder
for img_name in image_names:
    img_path = os.path.join('train_ims', img_name)
    image = imread(img_path)
    image_reversed = np.fliplr(image)
    reversed_img_name = 'reverse_' + img_name
    reversed_img_path = os.path.join('train_ims_rever', reversed_img_name)
    imsave(reversed_img_path, image_reversed)

print("All images have been horizontally flipped and saved to 'train_ims_rever' folder with 'reverse_' prefix.")

# Read the original label file
df = pd.read_csv('train.csv')

# Create a new label file for the flipped images
df_reversed = df.copy()
df_reversed['im_name'] = 'reverse_' + df_reversed['im_name']

# Combine the original and flipped labels
df_combined = pd.concat([df, df_reversed], ignore_index=True)
df_combined.to_csv('train_combined.csv', index=False)

print("Labels for flipped images have been added to 'train_combined.csv'.")

# Read the combined label file
df = pd.read_csv('train_combined.csv')
labels = df['label'].values
image_names = df['im_name'].values

# Encode the labels (10 categories)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Define image size and load images
IMG_SIZE = 128
channels = 3  # Grayscale; change to 3 if RGB
images = []
print("Start loading images...")
for img_name in image_names:
    if 'reverse_' in img_name:
        img_path = os.path.join('train_ims_rever', img_name)
    else:
        img_path = os.path.join('train_ims', img_name)
    image = cv2.imread(img_path)  # Grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    images.append(image)

images = np.array(images)
images = images.reshape(-1, IMG_SIZE, IMG_SIZE, channels) / 255.0  # Normalize
print("Images loaded, shape:", images.shape)

# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.17, random_state=42)
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
del images, df, df_reversed, df_combined, image_names
gc.collect()



Load environment successfully
All images have been horizontally flipped and saved to 'train_ims_rever' folder with 'reverse_' prefix.
Labels for flipped images have been added to 'train_combined.csv'.
Start loading images...
Images loaded, shape: (100000, 128, 128, 3)
Training set shape: (83000, 128, 128, 3)
Testing set shape: (17000, 128, 128, 3)


30

In [2]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Build a more powerful CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, channels)),
    data_augmentation,  # Add augmentation directly in the model
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(10, activation='softmax')  # 10 categories
])

# Compile with a lower learning rate for better convergence
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more epochs
print("Training CNN model...")
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

print("Model training completed")

# Evaluate on test set
print("Evaluating model on test set...")
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=1)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

del X_train, y_train
gc.collect()



2025-04-10 12:54:51.485249: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-04-10 12:54:51.485297: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-04-10 12:54:51.485301: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-04-10 12:54:51.485320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-10 12:54:51.485329: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Training CNN model...
Epoch 1/100


2025-04-10 12:59:34.362261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-10 12:59:34.370572: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


2335/2335 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4134 - loss: 1.7846

2025-04-10 13:01:50.315432: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


2335/2335 ━━━━━━━━━━━━━━━━━━━━ 140s 59ms/step - accuracy: 0.4134 - loss: 1.7845 - val_accuracy: 0.6061 - val_loss: 1.1140
Epoch 2/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 139s 59ms/step - accuracy: 0.5827 - loss: 1.1860 - val_accuracy: 0.6813 - val_loss: 0.8990
Epoch 3/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 138s 59ms/step - accuracy: 0.6443 - loss: 1.0072 - val_accuracy: 0.7134 - val_loss: 0.8154
Epoch 4/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 137s 59ms/step - accuracy: 0.6926 - loss: 0.8826 - val_accuracy: 0.7443 - val_loss: 0.7189
Epoch 5/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 142s 61ms/step - accuracy: 0.7181 - loss: 0.8107 - val_accuracy: 0.7754 - val_loss: 0.6414
Epoch 6/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 146s 63ms/step - accuracy: 0.7463 - loss: 0.7351 - val_accuracy: 0.7857 - val_loss: 0.6089
Epoch 7/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 156s 67ms/step - accuracy: 0.7647 - loss: 0.6774 - val_accuracy: 0.8054 - val_loss: 0.5572
Epoch 8/100
2335/2335 ━━━━━━━━━━━━━━━━━━━━ 153s 66ms/step - accuracy: 0.77

2025-04-10 16:01:59.155298: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


532/532 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step
Test Accuracy: 92.20%


1043